In [2]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 10)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
        
    def train(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

env = gym.make('CartPole-v1')
pi = Policy()
score = 0.0
print_interval = 20
for n_epi in range(10000):
    s, _ = env.reset()
    done = False
    while not done:
        prob = pi(torch.from_numpy(s).float())
        print(prob)
        m = Categorical(prob)
        print(m)
        a = m.sample()
        print(a)
        s_prime, r, done, truncated, info = env.step(a.item())
        pi.data.append((r,prob[a]))
        s = s_prime
        score += r
    pi.train()
    if n_epi%print_interval==0 and n_epi!=0:
        print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
        score = 0.0
env.close()

tensor([0.0962, 0.0985, 0.0935, 0.0960, 0.0941, 0.1044, 0.1097, 0.0901, 0.0971,
        0.1204], grad_fn=<SoftmaxBackward0>)
Categorical(probs: torch.Size([10]))
tensor(0)
tensor([0.0962, 0.1065, 0.0930, 0.0963, 0.0946, 0.1015, 0.1108, 0.0920, 0.0895,
        0.1197], grad_fn=<SoftmaxBackward0>)
Categorical(probs: torch.Size([10]))
tensor(6)


/econ_share/home/pp712/myenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


AssertionError: 6 (<class 'int'>) invalid